In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/nlp_tut/lesson7/отзывы за лето.xls'

In [ ]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 4.3 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [ ]:
data = pd.read_excel(path)

In [ ]:
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [ ]:
data['Rating'].unique()

array([5, 4, 2, 3, 1])

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = data['Content']
y = data['Rating']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.60, random_state=42)


In [ ]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, train_size = 0.5, random_state=42)

### Предобработка

In [ ]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=9650af488251751520be04c48b78e3b99572773c6b38577a68b995751783af0d
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 8.9 MB/s 


In [ ]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
  txt = str(txt)
  txt = "".join(c for c in txt if c not in exclude)
  txt = txt.lower()
  txt = re.sub("\sне", "не", txt)
  txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
  return " ".join(txt)

X_train = X_train.apply(preprocess_text)
X_val = X_val.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)

In [ ]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#### Отфильтруем данные и соберем в корпус N наиболее частых токенов

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
max_words = 200

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
import numpy as np

def text_to_sequence(text, maxlen):
  result = []
  tokens = word_tokenize(text.lower())
  tokens_filtered = [word for word in tokens if word.isalnum()]
  for word in tokens_filtered:
    if word in vocabulary:
      result.append(vocabulary[word])
  padding = [0]*(maxlen - len(result))
  return padding + result[-maxlen:]

In [ ]:
max_len = 40

In [ ]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in X_val], dtype=np.int32)

In [ ]:
x_train.shape

(12395, 40)

### Keras model

In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.metrics import categorical_crossentropy
from keras.callbacks import EarlyStopping

In [ ]:
import tensorflow as tf

In [ ]:
y_train

12170    5
15605    5
18144    5
8688     1
7541     5
        ..
11284    4
11964    5
5390     1
860      5
15795    5
Name: Rating, Length: 12395, dtype: int64

In [ ]:
num_classes = 6
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)

In [ ]:
y_train

array([[0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1.]], dtype=float32)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
tensorboard = TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
22/22 [==============================] - 10s 359ms/step - loss: 1.5370 - accuracy: 0.6524 - val_loss: 1.1929 - val_accuracy: 0.6952
Epoch 2/20
22/22 [==============================] - 7s 327ms/step - loss: 1.0493 - accuracy: 0.7017 - val_loss: 1.0131 - val_accuracy: 0.6952


In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score: ', score[0])
print('Test accuracy: ', score[1])

9/9 [==============================] - 1s 64ms/step - loss: 0.9757 - accuracy: 0.7021


Test score:  0.9757176637649536
Test accuracy:  0.7020813226699829


### 2.1 Инициализировать tf.keras.layers.Embedding предобученными весами rusvectores.org/ru/

In [ ]:
import tensorflow_hub as hub

In [ ]:
embedding = "/content/drive/MyDrive/nlp_tut/lesson7/news_0_300_2.bin"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)


OSError: ignored

In [ ]:
model.add(hub_layer)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))